# Exercise 1: Simple Inference

In this exercise, you will be running inference on a few images.
The primary goal of the first exercise is to familiarize you with the workflow for inference.

You will be creating a vehicle detection application where the model counts how many vehicles are found in an image. The image you will use is:

<img src="cars_1900_first_frame.jpg">

There appears to be 9 vehicles in the image. Let's see how the computer vision models do.

### Important! The quiz will ask you how many vehicles were detected in the last step.


## Step 1: Converting the Model

The first step is to get some compute vision models to use for the inference.
For this exercise,you will be using the following two models available from Model Downloader:

- `mobilenet-ssd`
- `vehicle-detection-adas-0002`

First, start with the `mobilenet-ssd`.
As the model downloader interface is terminal commands, the jupyter cell needs to be converted to run terminal commands (bash) by adding %%bash at the top.
The following cell has been populated with the downloader showing the help command. 

Modify the cell to download the `mobilenet-ssd` model.

Notes:
- You can download the model to anywhere you would like. But remember the path to the models, as you will need it later.

In [ ]:
%%bash
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --help

`mobilenet-ssd` model is a caffe model, so it needs to be converted to IR format. Furthermore, the scaling and the mean values must be set so that the network can take RGB images without needing to scale it in separate preprocessing code. 

These values should come from your expected dataset. For example if you are getting images from security cameras, the mean value should be the time average pixel value of your camera. But for this example set scale to 256 and mean values to \[127,127,127\].

Modify the following cell to convert the `mobilenet-ssd model` to IR format, with the image scale of 256 and image mean value of \[127,127,127\].

Notes:
- You can place the IR file anywhere but make sure to take note of the path.

In [ ]:
%%bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py --help

Now for the `vehicle-detection-adas-0002`.
In addition to the raw models like the `mobilenet-ssd`, model downloader has a number of models that are already converted to the IR format. 
`vehicle-detection-adas-0002` turns out to be one of them (in fact, this model is based on the same base network, mobilenet).
So for this model, you simply need to download it.


Modify the cell to download the `vehicle-detection-adas-0002` model.

Notes:
- Once again, you can place the IR file anywhere but make sure to take note of the path.

In [ ]:
%%bash
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --help

That's it! You now have the models needed to run inference.

## Step 2: Running Inference

Next step is running the inference itself.
This section will make use of the Inference Engine that we have covered. 
If you get stuck on any of the stps, refer to the slide deck from course 1 video 6.

First things first, run the following cell to import all the necessary Python modules.

In [ ]:
import matplotlib.pyplot as plt
import os                         
from openvino.inference_engine import IECore, IENetwork
import cv2

Start by creating the `IENetwork` objects.

Complete the following cell by creating `IENetwork` object for `mobilenet-ssd` model. You will need the path to the IR files from the earlier steps.

In [ ]:
# create IENetwork object for mobilenet-ssd

Next up is the `IECore` object. 

It turns out that the network has some layers that require a CPU extension. 
This can be verified using the result of query_network method of `IECore`, though you can skip this step in this exercise.
The required extension is one that is provided by the toolkit.

Complete the following cell by creating an `IECore` object, and add the specified `extension` to it.

In [ ]:
extension = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so'

Now for preprocessing. 

The input image will be loaded using OpenCV, so several image processing steps needs to be done.
First, it will have the wrong size. so the image must be reshaped using `cv2.resize()` function.
Second, OpenCV loads an image in HWC format whereas the network expects an NCHW format. 
So the image must first be transposed using `transpose()` method of numpy arrays. 
Then the N dimention must be added using the `reshape()` method.

As the preprocesisng is outside of the toolkit, they are already implemented for you.
However it is missing the sizes for dimensions NCHW of the network input.

Complete the `prepImage()` function by getting the values for `n`, `c`, `h` and `w` from the function input `net`.


In [ ]:
# Prepares image for imference
# inputs:
#     orig_image - numpy array containing the original, unprocessed image
#     net        - IENetwork object
# output: 
#     preprocessed image.
def prepImage(orig_image, net):
    
    ##! Find n, c, h, w from net !##
    
    # Resize the data to the input H and W
    input_image = cv2.resize(orig_image, (w, h))
    # Change the dimensions. old dim 2 -> new dim 0, old dim 0 -> new dim 1, and old dim 1 -> new dim 2
    input_image = input_image.transpose((2, 0, 1))
    # Add the N dimension
    input_image.reshape((n, c, h, w))

    return input_image

Finally the postprocessing. 

`mobilenet-ssd` returns 100 potential regions where an object might be.
For every potential object, the model assigns a probability that it is an object.
So to find the vehicles in the image, you need to look for entries over a certain threshold probability.
The model also provides bounding boxes for where the potential object is, and it returns an index to the detected object.
All this information can be processed and placed on the original image.

Postprocessing, however, is also outside of the toolkit.
So this step has already been implemented for you, and no modification is required.
The function takes in the numpy array result of the network (not the dictionary result) and the original image (before preprocessing). 
Additionally, the function set the object likelihood threshold to 50% but this can be overridden with `prob_threshold` argument.

Study the following cell and run it.

In [ ]:
# Processes the result. Prints the number of detected vehices and paints rectangles around the vehicles.
# inputs:
#    detected_obects - numpy array containing the ooutput of the model
#    orig_image      - numpy array containing the original, unprocessed image
#    prob_threashold - Required probability for "detection"
# output:
#    numpy array of image wtth rectangles drawn around the vehicles.
def showResult(detected_objects, orig_image, prob_threshold=0.5):
    initial_w = orig_image.shape[1]
    initial_h = orig_image.shape[0]
    out_frame = orig_image.copy()
    
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            xmin = int(obj[3] * initial_w)
            ymin = int(obj[4] * initial_h)
            xmax = int(obj[5] * initial_w)
            ymax = int(obj[6] * initial_h)
            class_id = int(obj[1])
            # Draw box and label\class_id
            color = (255,255,255)
            cv2.rectangle(out_frame, (xmin, ymin), (xmax, ymax), color, 2)
            cv2.putText(out_frame, str(round(obj[2] * 100, 1)) + ' %', (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 0.6, color, 1)
            detected_count+=1
    print("{} vehicles detected.".format(detected_count))
    fig = plt.figure(dpi=300)
    ax = fig.add_subplot(111)
    ax.imshow(cv2.cvtColor(out_frame, cv2.COLOR_BGR2RGB), interpolation='none')
    plt.axis("off")
    plt.show()
    del out_frame

Now to put everything together and run an inference workload on an image. 

An image has been provided to you, but you need to preprocess it using the `prepImage()` function from earlier. It takes the raw image as well as the IENetwork you created earlier.
Hint: Be sure not to overwrite the original image, as you will need it for `showResults()`


In [ ]:
image_path = "cars_1900_first_frame.jpg"
original_image = cv2.imread(image_path)
# Preprocess the image.

Next, create an ExecutableNetwork object for CPU device using the IENetwork object for `mobilenet-ssd`.

In [ ]:
# Create ExecutableNetwork object.

Now run the inference in the synchronous mode. Remember that you will need the name of the input layer.

In [ ]:
# You need the name of input layer. There is only one input layer.

# Run synchronous inference.

Finally, process the result by running the `showResult()` function. Note that this function  accepts an array result, so you need to extact the array from the dictionary result of the inference.

In [ ]:
# You need the name of output layer. There is only one output layer.

# Run showResult. Make sure you extracted the array result from the dictionary returned by the inference.

Congratulations! You have successfully run a computer vision application with the toolkit. This workload was done on the CPU, so now let's try it on other devices.

## Step 3: Using the job queue and the DevCloud

Next we will look at how to run inference on other types of nodes. 
But to do this, we need to discuss the job queue feature.

DevCloud relies on a job queue to give fair access to the available resources.
Users can submit "jobs" to the queue and request a certain type of node (e.g. system with VPU) to complete the task.
The job scheduler will then take the job and execute it on the first available system of that type.
In addition to special hardware type, the jobs on nodes will have far more resources in terms of RAM and CPU than this Jupyter notebook. 
So any workload beyond simple testing should be executed through the job queue.

The resource manager used in the DevCloud is called PBS Torque, and you can interact with it using commands prefixed with 'q'.
The DevCloud website has instructions on how to use these tools.
For this course however, the commands are provided to you.

The main difference for the purpose of this exercise is that you can not simply "run" jupyter cells on the queue.
Instead you need to create python files to be executed.
In jupyter, cells beginning with `%%writefile myfile.txt` will dump the contents of the cell into a file, named myfile.txt in this case.
Below cell has an incomplete `main.py` file for running the same workload.

The instructions for completing the file is broken into steps. 
In the cell, the parts that need to be modified is signified by `##! ... !##`
The number in parenthesis shows the step in the instruction that this corresponds to.
Follow the instructions to complete `main.py`. (Hint: most of these should be a simple copy and paste from earlier cells)

*(2.1)* Complete the `prepImage()` function by finding the NCHW values from the network. 

*(2.2)* Create IENetwork models for `vehicle-detection-adas-0002`. 

*(2.3)* Create IECore object and load the extension. Optional: in this script, the device to use is an user input and is stored in `device` variable. Add an if check to see if `device` includes CPU. With that said, it is safe to load the CPU extension even if we do not use CPU for the inference. 

*(2.4)* Preprocess the image with `prepImage()`.

*(2.5)* Create an ExecutableNetwork object. IENetwork should be the one created earlier, and the device should be the one in `device` variable. This variable is set by the commandline input to the main.py script.

*(2.6)* Run synchronous inference. Remember that you will need the name of the input layer.

*(2.7)* Run printCount. This is the equivalent of showResult from earlier, but does not show the image. Remember that you need the output layer name. 

Side Note: Jobs that we submit to job queues will not have HTML outputs, so this example simply prints the number of detected cars. If you need the image, you will need to write it into a file. 


In [ ]:
%%writefile main.py
import os
import sys
from openvino.inference_engine import IECore, IENetwork
import cv2

# Prepares image for imference
# inputs:
#     orig_image - numpy array containing the original, unprocessed image
#     net        - IENetwork object
# output: 
#     preprocessed image.
def prepImage(orig_image, net):
    
    ##! (2.1) Find n, c, h, w from net !##
    
    input_image = cv2.resize(orig_image, (w, h))
    input_image = input_image.transpose((2, 0, 1))
    input_image.reshape((n, c, h, w))
    return input_image

# Processes the result. Prints the number of detected vehices.
# inputs:
#    detected_obects - numpy array containing the ooutput of the model
#    prob_threashold - Required probability for "detection"
# output:
#    numpy array of image wtth rectangles drawn around the vehicles.
def printCount(detected_objects, prob_threshold=0.5):
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            detected_count+=1    
    print("{} vehicles detected.".format(detected_count))

# Getting the device as commandline argument
device = sys.argv[1]
    
##! (2.2) create IENetwork object for vehicle-detection-adas-0002 !##

##! (2.3) create IECore object and load the following extension !##
extension = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so'

image_path = "cars_1900_first_frame.jpg"
original_image = cv2.imread(image_path)

##! (2.4) Preprocess the image. !##

##! (2.5) Create ExecutableNetwork object. Use the device variable for targetted device !##

##! (2.6) Run synchronous inference. !##

##! (2.7) Run printCount. Make sure you extracted the array result form the dictionary returned by infer(). !##

Now that you have the script, you are ready to submit this workload to the queue.

Once in the queue, the workload will be executed on a remote system and the output is returned to you in a file.
This process can take some time, so we have provided an utility function that automatically waits for completion.
The details of this function are not directly relevant to the exercise, but you can find the code in [notebook_utils.py](notebook_utils.py)

Jobs are submitted through the `qsub` command, and the command you need is provided in the following cell.
For the purposes of this lab, there are two important details.
One is the command in the quotes, `python3 main.py CPU` is the command that is executed on the system.
The second is the `-l` flag that specifies what type of node the job requests.
If you want to learn more about using the DevCloud queue, go to the DevCloud documentation page.

Run the following cell to run the job on the CPU.

In [ ]:
from notebook_utils import waitForJob
job_name = !echo "python3 main.py CPU" | qsub -d `pwd` -N objdet -l nodes=1:skylake
print("Job submitted. Waiting for output. This may take some time.")
waitForJob(job_name)

Now run the following cell to run the job on the GPU.

In [ ]:
job_name = !echo "python3 main.py GPU" | qsub -d `pwd` -N objdet -l nodes=1:skylake:intel-hd-530
print("Job submitted. Waiting for output. This may take some time.")
waitForJob(job_name)

Congratulations! You have successfully run the inference workload on a GPU and a CPU. 

## Step 4: Quiz question

For the final step, let's try lowering the required confidence to 0.01 (e.g. 1%) and see how many vehicles are detected by the model. 
You will have to go back to the `main.py` cell, and add `prob_threshold` argument to `printCount` function.
**The quiz will ask you how many vehicles were detected by the vehicle-detection-adas-0002 at this setting.**